In [6]:
import pandas as pd
from methods import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
def select_columns_from_dataset():
    df = pd.read_csv(r'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-30-random_filters.csv')
#     for col_to_delete in ['square30_r_mean_diff', 'square30_r_std_diff', 'square30_r_mean', 'square30_r_std']:
    for col_to_delete in ['square30_r_mean_diff', 'square30_r_std_diff', 'square30_r_mean', 'square30_r_std']:
        del df[col_to_delete]
    df.to_csv(r'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_filters.csv', index=False)

def merge_datasets():
#     for size in [10, 20, 25, 30, 35, 40, 45, 50]:
#     for size in [25, 30, 35, 40, 45, 50]:
    for size in [30]:
#         path = fr'confusion-reports\ics_fc\round3-train-dataset\round3-train-dataset_fc_square-{size}-random_filters.csv'
        path = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-{size}_filters.csv'
        df_to_modify_and_save = pd.read_csv(path)
        df_to_copy_columns_from = pd.read_csv(fr'confusion-reports\round3-columns.csv')
        df_to_modify_and_save['architecture_code'] = df_to_copy_columns_from['architecture_code']
#         for col in [f'square{size}_mean_diff', f'square{size}_std_diff', f'square{size}_mean', f'square{size}_std']:
#             df_to_modify_and_save[col] = df_to_copy_columns_from[col]
        new_order_for_columns = [
            'model_name', 'model_architecture', 'architecture_code', 'model_label', 'trigger_type_aux',

            f'square{size}_mean_diff', f'square{size}_std_diff',
#             f'square{size}_r_mean_diff', f'square{size}_r_std_diff',
            'gotham_mean_diff', 'gotham_std_diff',
            'kelvin_mean_diff', 'kelvin_std_diff',
            'lomo_mean_diff', 'lomo_std_diff',
            'nashville_mean_diff', 'nashville_std_diff',
            'toaster_mean_diff', 'toaster_std_diff',

            'clean_mean', 'clean_std',
            f'square{size}_mean', f'square{size}_std',
#             f'square{size}_r_mean', f'square{size}_r_std',
            'gotham_mean', 'gotham_std',
            'kelvin_mean', 'kelvin_std',
            'lomo_mean', 'lomo_std',
            'nashville_mean', 'nashville_std',
            'toaster_mean', 'toaster_std',

            'trigger_color', 'num_classes',
        ]
        df_to_modify_and_save = df_to_modify_and_save[new_order_for_columns]
        df_to_modify_and_save.to_csv(path, index=False)

def correct_column_in_dataset():
    df_with_correct_column = pd.read_csv(r'confusion-reports\ics_svm\round3-train-dataset\round3-columns.csv')
    column = 'trigger_type_aux'
    for size in [10, 15, 20, 25,30, 35, 40, 45, 50]:
        file = fr'confusion-reports\ics_svm\round3-train-dataset\round3-train-dataset_square-{size}-filters_all-classes_gray.csv'
        df_to_be_corrected = pd.read_csv(file)
        df_to_be_corrected[column] = df_with_correct_column[column]
        df_to_be_corrected.to_csv(file, index=False)

def create_architecture_code():
    df_columns = pd.read_csv(r'confusion-reports\round3-columns.csv')
    list_codes = []
    for i in range(len(df_columns)):
        model_architecture = df_columns['model_architecture'].iloc[i]
        arch_code = encode_architecture(model_architecture)
        if arch_code is None:
            print(f'Error at index {i}')
            break
        list_codes.append(arch_code)
    df_columns['architecture_code'] = list_codes
    df_columns = df_columns[['model_name', 'poisoned', 'model_architecture', 'architecture_code', 'trigger_type_aux', 'trigger_type', 'instagram_filter_type', 'polygon_side_count']]
    df_columns.to_csv(r'confusion-reports\round3-columns.csv', index=False)
    
merge_datasets()
# select_columns_from_dataset()
# correct_column_in_dataset()
# create_architecture_code()
print('done')

done
